In [ ]:
import cv2
import numpy as np
import pywt
import os
from scipy.ndimage import uniform_filter

 ### 🧮 Cálculo subbandas diagonales con wavelet

Esta función `obtener_subbandas_diagonales` aplica la transformada de wavelet a una imagen en formato YCbCr y se queda con las subbandas diagonales (resultado de aplicar dos filtros de paso alto) de cada canal.

**¿Cómo funciona?**
- Separa la imagen en los 3 canales con `cv2.split(imagen_ycbcr)`
- Primero calcula la transformada de wavelet de la imagen con `pywt.wavedec2()`. Esto devuelve aproximación (LL), subbandas verticales (LH), subbandas horizontales (HL) y subbandas diagonales (HH)
- De estos 4 coeficientes, me quedo sólo con el diagonal.

In [ ]:

def obtener_subbandas_diagonales(imagen_ycbcr, wavelet='db1'):
    """
    Aplica la transformada wavelet a cada canal de la imagen YCbCr y extrae las subbandas diagonales (HH).
    """
    subbandas_HH = []
    canales = cv2.split(imagen_ycbcr)                                # Separar los canales Y, Cb, Cr 
    for canal in canales:                                            #Recorro los canales 
        coeficientes = pywt.wavedec2(canal, wavelet=wavelet, level=1)# Wavedec2 descompone la imagen en coeficientes de aproximación (LL), horizontal(HL), vertical(LH) y diagonal (HH)
        _, *detalles = coeficientes                                  # Extraer la subbanda diagonal (HH) en el nivel deseado
        HH = detalles[0][2]                                          # Accedemos a la subbanda HH en el nivel seleccionado
        subbandas_HH.append(HH)
    
    return subbandas_HH

 ### 🧮 Cálculo matrices MH_i

Esta función `obtener_MH_i` obtiene matrices MH_i que eliminan el sesgo del contenido de alta frecuencia, "limpia" distorsiones o influencias no deseadas que provengan de las altas frecuencias de la imagen.

**¿Cómo funciona?**
- Se le pasa como parámetro una matriz de alta frecuencia (subbanda diagonal)
- Calcula la media de cada bloque no solapado con la funcion auxiliar `blockwise_mean()`
- A la matriz con el contenido de alta frecuencia se le resta la media de esos bloques y se queda con el valor absoluto para obtener la nueva matriz

In [ ]:
def blockwise_mean(HH_i, block_size):
    """Calcula la media por bloques no solapados."""
    h, w = HH_i.shape
    bs = block_size
    # Recortar imagen para que encaje con múltiplos del block size
    h_crop, w_crop = h - h % bs, w - w % bs
    img_cropped = HH_i[:h_crop, :w_crop]
    # Redimensionar y calcular la media por bloque
    blocks = img_cropped.reshape(h_crop // bs, bs, w_crop // bs, bs).mean(axis=(1, 3))
    # Expandir cada valor a su bloque original
    expanded = np.kron(blocks, np.ones((bs, bs)))  # expandir para restar luego
    # Rellenar para igualar tamaño original (útil en caso de que el tamaño de la imagen no sea dibisible por el tamaño del bloque)
    block_mean_full = np.zeros_like(HH_i)
    block_mean_full[:h_crop, :w_crop] = expanded
    return block_mean_full

def obtener_MH_i(HH_i, block_size):
    """ Calcula MH para una subbanda de alta frecuencia H, restando la media de bloques NO solapados."""
    high_freq = HH_i
    block_mean = blockwise_mean(high_freq, block_size)
    return np.abs(high_freq - block_mean)

### 🧮 Cálculo matrices S_i

Esta función `obtener_S_i` calcula la desviación estándar local con el mismo tamaño de bloque para todo los bloques solapados.

**¿Cómo funciona?**
- Se le pasa como parámetro la matriz con el contenido de alta frecuencia 
- Calcula el promedio de los valores dentro de una ventana de tamaño "block_size" alrededor de un pixel. Cada uno de esos valores promedio se guardan en la matriz "media"
- Repite con los valores de la matriz original al cuadrado
- Calcula y devuelve una matriz donde cada valor reprensenta la desvaiación estándar local de cada bloque

In [ ]:
def obtener_S_i(HH_i, block_size):
    """Calcula la desviación estándar local de bloques solapados."""
    media = uniform_filter(HH_i, size=block_size)  
    media_sq = uniform_filter(HH_i**2, size=block_size)
    return np.sqrt(np.maximum(media_sq - media**2, 0))

### 🧮 Cálculo matriz T_canal

Esta función `calcular_matriz_T` calcula la matriz T ayudándose de la matriz con el contenido de alta frecuencia "MH_i" y la matriz con la desviación estándar local "S_i".

**¿Cómo funciona?**
- Se le pasan ambas matrices como parámetros
- Calcula el numerador y denominador según las fórmulas (4), (5) y (6) del artículo científico "High Frequency Content based Stimulus for Perceptual Sharpness Assessment in Natural Images"
- Devuelve el resultado

In [ ]:
def calcular_matriz_T(MH_I, S_I, alpha=1):
    """Calcula la matriz T dada MH_I y S_I"""
    numerador = (MH_I ** alpha) * S_I
    denominador = np.sum(S_I) if np.sum(S_I) != 0 else 1.0

    return numerador / (denominador + 1e-8)  # evitar división por cero

### 🧮 Obtener las matrices T de cada canal

Esta función `obtener_matrices_T` calcula la matriz T de cada uno de los canales usando las funciones anteriormente descritas.

**¿Cómo funciona?**
- Se le pasa el contenido de alta frecuencia como parámetro
- Calcula la matriz MH_i y S_i
- Calcula la matriz T de cada canal
- Devuelve las 3 matrices T

In [ ]:
def obtener_matrices_T(SubbandasDiagonales, block_size = (8, 8), alpha=1):
    """
    Aplica la fórmula a cada canal para obtener TY, TCb, TCr.
    """
    T_out = []
    for i in range(3):
        #HH_subbandas[i] = HH_i.astype(np.float32)
        MH_i = obtener_MH_i(SubbandasDiagonales[i], block_size) # 1) Calcular MH (bloques no ) 
        S_i = obtener_S_i(SubbandasDiagonales[i], block_size)   # 2) Calcular S (bloques superpuestos)      
        T_i = calcular_matriz_T(MH_i, S_i, alpha)               # 3) Calcular T
        T_out.append(T_i)

    return T_out[0], T_out[1], T_out[2]  #Devuelve Ty, Tcb, Tcr

### 🧮 Obtener estímulo total T

Esta función `calcular_estimulo_total` hace la media de las matrices T de cada canal.

**¿Cómo funciona?**
- Se le pasan las matrices T de cada canal de la imagen
- Hace la media y la devuelve

In [ ]:
def calcular_estimulo_total(TY, TCb, TCr, alpha=1):
    """
    Calcula el total stimulus T_S de la imagen usando la ecuación dada.
    """
    suma = (TY + TCb + TCr) / 3     # Sumar las matrices de los tres canales
    T_S = np.power(suma, 1/alpha)   # Calcular el total stimulus T_S
    
    return T_S

### 🧮 Obtener mapa de nitidez bruto

Esta función `calcular_Smap` calcula el mapa de nitidez bruto de la imagen aplicando la fórmula (8) del artículo científico "High Frequency Content based Stimulus for Perceptual Sharpness Assessment in Natural Images" 

**¿Cómo funciona?**
- Se le pasa el estímulo total como parámetro
- Aplica la fórmula y devuelve el mapa 


In [ ]:
def calcular_Smap(T_S, epsilon=1e-6):
    """
    Calcula la matriz Smap (mapa de nitidez) utilizando la ecuación dada:
    Smap(i, j) = abs[log(ε) + ε] / (abs[log(T_S(i, j)) + ε] + ε)
    """
    # Numerador: abs(log(epsilon) + epsilon)
    numerador = np.abs(np.log(epsilon) + epsilon)
    denominador = np.abs(np.log(T_S + epsilon)+ epsilon) # Denominador: abs(log(T_S) + epsilon) + epsilon
    # Calcular Smap
    Smap = numerador / denominador
    return Smap

### 🧮 Obtener mapa de nitidez sin bordes

Esta función `obtener_BSmap` descarta los bordes del mapa de nitidez (en función del tamaño del bloque) para eliminar el efecto borde; a este nuevo mapa se le llama bsmap

**¿Cómo funciona?**
- Se le pasa como parámetro el mapa de nitidez
- Calcula el tamaño del borde a eliminar según el tamaño del bloque
- Elimina ese borde y devuelve el mapa

In [ ]:
def obtener_BSmap(Smap, block_size=8):
    """
    Elimina el borde del mapa de nitidez (Smap) en función del tamaño del bloque.
    """
    # El tamaño del borde a eliminar es block_size - 1 en cada dirección
    border_size = block_size - 1
    # Recortar el borde del Smap
    BSmap = Smap[border_size:-border_size, border_size:-border_size] if border_size > 0 else Smap

    return BSmap

### 🧮 Obtener puntuación de calidad

Esta función `obtener_calidad_imagen` obtiene la calidad de la nitidez de la imagen cogiendo el máximo valor de BSmap, el cual ha sido calculado con las funciones anteriores del notebook

**¿Cómo funciona?**
- Se le pasa como parámetro una imagen en YCbCr
- Se obtienen las subbandas diagonales de cada canal mediante wavelet
- Obtiene el estimulo total a través de las matrices T de cada canal 
- Calcula el mapa de nitidez (Smap) y le quita los bordes (BSmap)
- Obtiene el valor máximo de BSmap y lo devuelve

In [ ]:
def obtener_calidad_imagen(imagen_ycbcr, block_size = 8, epsilon=1e-6, alpha=1, wavelet='db1'):
    
    SubbandasDiagonales = obtener_subbandas_diagonales(imagen_ycbcr, wavelet)
    TY, TCb, TCr = obtener_matrices_T(SubbandasDiagonales, block_size , alpha)
    T_S = calcular_estimulo_total(TY, TCb, TCr, alpha)
    Smap = calcular_Smap(T_S, epsilon)
    BSmap = obtener_BSmap(Smap, block_size=8)
    Qs = np.max(BSmap)
    return Qs

### 🗂️ Procesamiento masivo de imágenes con wavelet

Esta función `procesar_imagenes_carpeta_wavelet` obtiene la puntuación de calidad siguiendo el método de obtención de contenido de alta frecuencia en una imagen usando la transformada de wavelet en todas la imagenes `.png` dentro de una estructura de carpetas. 

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Genera la puntuacion de calidad de cada imagen.
- Guarda la puntuacion de calidad de cada imagen en un archivo `.txt`.

In [ ]:
def procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    wavelet='db1' #Tipo de wavelet
):
    """
    Recorre todas las subcarpetas dentro de 'carpeta_entrada'.
    - Para cada archivo .png, aplica la transformada de wavelet
    - Guarda la imagen resultante en 'carpeta_salida', manteniendo la misma estructura.
    - Genera un archivo .txt con el valor de nitidez de cada imagen procesada.

    Parámetros:
    - carpeta_entrada: ruta de la carpeta de entrada.
    - carpeta_salida: ruta de la carpeta donde se guardarán los resultados.
    - wavelet: especifica el tipo de wavelet que se aplica, por defecto Daubechies de 1 nivel.
    
    """
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'wavelet'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "wavelet")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar las varianzas
        ruta_txt = os.path.join(carpeta_salida_sub, "info_nitidez_wavelet.txt")
        
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Calidad de la imagen usando la (Transformada de wavelet)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Un valor mayor corresponde a una mayot nitidez.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(("sharp.png", "blur.png", "blur_gamma.png")):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en YCbCr
                    img_bgr = cv2.imread(ruta_imagen_entrada)
                    if img_bgr is None:
                        continue
                    img_YCbCr = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
                    
                    # Obtengo BSmap y quality score
                    Qs= obtener_calidad_imagen(img_YCbCr, wavelet=wavelet)
                
                    archivo_txt.write(f"{filename} -> Quality score: {Qs:.3f}\n")

    print("Procesamiento completado con transformada de wavelet.")

In [ ]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
)